In [3]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [23]:
def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(json_url, cache_path))
    return df

In [24]:
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015
end_date = datetime.now() # up until today
pediod = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

In [25]:
altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','SC','XMR','XEM']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1420070400.0&end=1588346474.229236&period=86400 at BTC_ETH
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1420070400.0&end=1588346474.229236&period=86400 at BTC_LTC
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XRP&start=1420070400.0&end=1588346474.229236&period=86400 at BTC_XRP
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETC&start=1420070400.0&end=1588346474.229236&period=86400 at BTC_ETC
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_STR&start=1420070400.0&end=1588346474.229236&period=86400 at BTC_STR
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_DASH&start=1420070400.0&end=1588346474.229236&period=86400 at BTC_DASH
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_SC&start=1420070400.0&end=1588346474.229236&period=8

In [26]:
altcoin_data['ETH'].tail()

,high,low,open,close,volume,quoteVolume,weightedAverage
date,,,,,,,
2020-04-27,0.025684,0.024864,0.025660,0.025270,584.050468,23062.613551,0.025325
2020-04-28,0.025476,0.025100,0.025266,0.025364,446.631411,17648.927039,0.025306
2020-04-29,0.026000,0.024248,0.025357,0.024536,955.077648,38072.413926,0.025086
2020-04-30,0.024654,0.023612,0.024531,0.023903,1395.280064,58068.592759,0.024028
2020-05-01,0.024363,0.023865,0.023898,0.024087,459.752395,19079.081584,0.024097


In [28]:
btc_usd_datasets = get_crypto_data('USDT_BTC')
btc_usd_datasets.tail()

Cached https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1420070400.0&end=1588346474.229236&period=86400 at USDT_BTC


,high,low,open,close,volume,quoteVolume,weightedAverage
date,,,,,,,
2020-04-27,7788.000000,7624.601700,7694.754048,7773.240743,2.421146e+07,3143.131254,7702.975600
2020-04-28,7777.902604,7661.413123,7776.851464,7741.740465,1.688243e+07,2187.180908,7718.806164
2020-04-29,8946.512400,7709.533511,7741.740465,8777.187162,5.380971e+07,6461.380467,8327.896334
2020-04-30,9454.747612,8405.000000,8777.187162,8624.959057,5.046225e+07,5662.458749,8911.720146
2020-05-01,9056.666670,8617.955700,8624.959057,8826.000000,1.894907e+07,2146.261962,8828.869200


In [30]:
# Calculate USD Price as a new column in each altcoin dataframe
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] =  altcoin_data[altcoin]['weightedAverage'] * btc_usd_datasets['weightedAverage']

In [32]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

In [33]:
# Merge USD price of each altcoin into single dataframe 
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')

In [34]:
# Add BTC price to the dataframe
combined_df['BTC'] = btc_usd_datasets['weightedAverage']

In [38]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            #visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In [39]:
# Chart all of the altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (USD)', seperate_y_axis=False, y_axis_label='Coin Value (USD)', scale='log')

In [40]:
# Calculate the pearson correlation coefficients for cryptocurrencies in 2016
combined_df_2016 = combined_df[combined_df.index.year == 2016]
combined_df_2016.pct_change().corr(method='pearson')

,ETH,LTC,XRP,ETC,STR,DASH,SC,XMR,XEM,BTC
ETH,1.000000,-0.048222,0.074841,-0.186078,0.032918,0.115291,0.162705,0.086184,0.056042,0.014352
LTC,-0.048222,1.000000,0.069180,-0.134321,0.134052,0.021657,0.010476,0.146890,0.200024,0.778171
XRP,0.074841,0.069180,1.000000,-0.060962,0.318903,0.077461,0.010986,0.026656,0.118661,0.066592
ETC,-0.186078,-0.134321,-0.060962,1.000000,-0.108291,-0.005872,-0.012750,-0.107600,-0.083462,-0.171296
STR,0.032918,0.134052,0.318903,-0.108291,1.000000,0.059207,0.136886,0.032431,0.238921,0.107238
DASH,0.115291,0.021657,0.077461,-0.005872,0.059207,1.000000,0.015329,0.124527,0.038086,0.033544
SC,0.162705,0.010476,0.010986,-0.012750,0.136886,0.015329,1.000000,0.043329,0.110965,0.033084
XMR,0.086184,0.146890,0.026656,-0.107600,0.032431,0.124527,0.043329,1.000000,0.029528,0.148529
XEM,0.056042,0.200024,0.118661,-0.083462,0.238921,0.038086,0.110965,0.029528,1.000000,0.267388
BTC,0.014352,0.778171,0.066592,-0.171296,0.107238,0.033544,0.033084,0.148529,0.267388,1.000000


In [43]:
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').to_numpy(),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [44]:
correlation_heatmap(combined_df_2016.pct_change(), "Cryptocurrency Correlations in 2016")

In [45]:
combined_df_2019 = combined_df[combined_df.index.year == 2019]
combined_df_2019.pct_change().corr(method='pearson')

,ETH,LTC,XRP,ETC,STR,DASH,SC,XMR,XEM,BTC
ETH,1.000000,0.770677,0.753526,0.724699,0.694725,0.789073,0.643893,0.769026,0.674704,0.768593
LTC,0.770677,1.000000,0.659868,0.629547,0.621005,0.674447,0.590909,0.691497,0.555291,0.679186
XRP,0.753526,0.659868,1.000000,0.640838,0.749073,0.684374,0.529650,0.679552,0.572210,0.623018
ETC,0.724699,0.629547,0.640838,1.000000,0.604289,0.623128,0.581325,0.631667,0.585884,0.588756
STR,0.694725,0.621005,0.749073,0.604289,1.000000,0.643127,0.540611,0.612716,0.653004,0.502858
DASH,0.789073,0.674447,0.684374,0.623128,0.643127,1.000000,0.619159,0.756943,0.568218,0.720753
SC,0.643893,0.590909,0.529650,0.581325,0.540611,0.619159,1.000000,0.603644,0.558530,0.609578
XMR,0.769026,0.691497,0.679552,0.631667,0.612716,0.756943,0.603644,1.000000,0.553590,0.738075
XEM,0.674704,0.555291,0.572210,0.585884,0.653004,0.568218,0.558530,0.553590,1.000000,0.530515
BTC,0.768593,0.679186,0.623018,0.588756,0.502858,0.720753,0.609578,0.738075,0.530515,1.000000


In [46]:
correlation_heatmap(combined_df_2019.pct_change(), "Cryptocurrency Correlations in 2019")